In [ ]:
import os, sys
import pandas as pd
import geopandas as gpd
from dotenv import load_dotenv

sys.path.append('../')
from DataService import DataService

In [ ]:
SOIL_TABLE = 'soil_map'
CMP_SOIL_TABLE = 'soil_cmp'
CRT_SOIL_TABLE = 'soil_crt'
EFT_SOIL_TABLE = 'soil_eft'
LAT_SOIL_TABLE = 'soil_lat'
LDT_SOIL_TABLE = 'soil_ldt'
LET_SOIL_TABLE = 'soil_let'
LST_SOIL_TABLE = 'soil_lst'
MB_SOIL_TABLE = 'mb_soil_data'
AB_SOIL_TABLE = 'ab_soil_data'
SK_SOIL_TABLE = 'sk_soil_data'

load_dotenv()
PG_DB = os.getenv('POSTGRES_DB')
PG_ADDR = os.getenv('POSTGRES_ADDR')
PG_PORT = os.getenv('POSTGRES_PORT')
PG_USER = os.getenv('POSTGRES_USER')
PG_PW = os.getenv('POSTGRES_PW')

In [ ]:
db = DataService(PG_DB, PG_ADDR, PG_PORT, PG_USER, PG_PW)
conn = db.connect()

In [ ]:
soilMapFile = "data/ca_all_slc_v3r2.shp"
soilMap = gpd.read_file(soilMapFile, encoding='utf-8')
soilMap = soilMap.set_crs(crs='EPSG:4326', allow_override=True)
soilMap = soilMap.to_crs(crs='EPSG:3347')

file = "data/ca_all_slc_v3r2_cmp.dbf"
cmp = gpd.read_file(file, encoding='utf-8')
cmp = pd.DataFrame(cmp.drop(columns='geometry'))

file = "data/ca_all_slc_v3r2_crt.dbf"
crt = gpd.read_file(file, encoding='utf-8')
crt = pd.DataFrame(crt.drop(columns='geometry'))

file = "data/ca_all_slc_v3r2_eft.dbf"
eft = gpd.read_file(file, encoding='utf-8')
eft = pd.DataFrame(eft.drop(columns='geometry'))

file = "data/ca_all_slc_v3r2_lat.dbf"
lat = gpd.read_file(file, encoding='utf-8')
lat = pd.DataFrame(lat.drop(columns='geometry'))

file = "data/ca_all_slc_v3r2_ldt.dbf"
ldt = gpd.read_file(file, encoding='utf-8')
ldt = pd.DataFrame(ldt.drop(columns='geometry'))

file = "data/ca_all_slc_v3r2_let.dbf"
let = gpd.read_file(file, encoding='utf-8')
let = pd.DataFrame(let.drop(columns='geometry'))

file = "data/ca_all_slc_v3r2_lst.dbf"
lst = gpd.read_file(file, encoding='utf-8')
lst = pd.DataFrame(lst.drop(columns='geometry'))

file = "data/soil_name_mb_v2r20130705.dbf"
mbSoil = gpd.read_file(file, encoding='utf-8')
mbSoil = pd.DataFrame(mbSoil.drop(columns='geometry'))

file = "data/soil_name_ab_v2r20140529.dbf"
abSoil = gpd.read_file(file, encoding='utf-8')
abSoil = pd.DataFrame(abSoil.drop(columns='geometry'))

file = "data/soil_name_sk_v2r20130705.dbf"
skSoil = gpd.read_file(file, encoding='utf-8')
skSoil = pd.DataFrame(skSoil.drop(columns='geometry'))

In [ ]:
soilMap.to_postgis(SOIL_TABLE, conn, index=False, if_exists='replace')

cmp.to_sql(CMP_SOIL_TABLE, conn, schema='public', if_exists="append", index=False)
crt.to_sql(CRT_SOIL_TABLE, conn, schema='public', if_exists="append", index=False)
eft.to_sql(EFT_SOIL_TABLE, conn, schema='public', if_exists="append", index=False)
lat.to_sql(LAT_SOIL_TABLE, conn, schema='public', if_exists="append", index=False)
ldt.to_sql(LDT_SOIL_TABLE, conn, schema='public', if_exists="append", index=False)
let.to_sql(LET_SOIL_TABLE, conn, schema='public', if_exists="append", index=False)
lst.to_sql(LST_SOIL_TABLE, conn, schema='public', if_exists="append", index=False)

mbSoil.to_sql(MB_SOIL_TABLE, conn, schema='public', if_exists="append", index=False)
abSoil.to_sql(AB_SOIL_TABLE, conn, schema='public', if_exists="append", index=False)
skSoil.to_sql(SK_SOIL_TABLE, conn, schema='public', if_exists="append", index=False)

In [ ]:
db.cleanup()